In [1]:
import os 
from torch.utils import *
from fastai2.vision.widgets import *
from fastai2.vision.all import *
import matplotlib.pyplot as plt

In [2]:
learn=load_learner('export.pkl')
upload_button=widgets.FileUpload()
btn_run = widgets.Button(description='Find out Now!!')
lbl_pred = widgets.Label()
out_pl = widgets.Output()

   #                 Which Celebrity do you look like? 

   ###                          Find out by uploading a selfie or a picture!

In [3]:
def upload(): 
    uploaded_img = PILImage.create(upload_button.data[-1])
    resize_img=uploaded_img.reshape(256,256)
    img_tensor=image2tensor(resize_img)
    return uploaded_img,resize_img,img_tensor

In [4]:
def find_similar_image(img_tensor,pred):
    path=Path('data/data')/pred
    mat=[]
    for file in os.listdir(path):
        if (file=='.ipynb_checkpoints'):
            continue
        image = PILImage.create(path/file)
        image=image.reshape(256,256)
        comp_tensor=image2tensor(image)
        mat.append(np.sum((img_tensor.numpy()-comp_tensor.numpy())**2))
    compare_img=PILImage.create(path/os.listdir(path)[np.argmin(mat)]).reshape(256,256)
    return compare_img

In [5]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

In [6]:
def on_click_classify(change):
    uploaded_img,resize_img,img_tensor = upload() 
    out_pl.clear_output()
#     with out_pl: display(uploaded_img.to_thumb(256,256))
    pred,pred_idx,probs = learn.predict(uploaded_img)
    compare_img=find_similar_image(img_tensor,pred)
    with out_pl: display(get_concat_h(resize_img,compare_img))
    lbl_pred.value = f'You look like: {pred}!' #; Probability: {probs[pred_idx]:.04f}'
btn_run.on_click(on_click_classify)

In [7]:
VBox([widgets.Label(''), 
      upload_button, btn_run, out_pl, lbl_pred])